# **Multimodal Machine Learning with Brain, Image, and Text Data**

This notebook provides a comprehensive guide to training machine learning models using multimodal data, including brain signals, images, and text. You will explore various data processing techniques, model training using neural networks and traditional classifiers, and performance evaluation using metrics like accuracy, precision, and recall. The goal is to integrate diverse data sources to improve classification performance and gain deeper insights into complex datasets.

# **Install the package**

##**1. Install the data reading package**

**Library Installation**:

The code first installs the required library, mmbra, using the command pip install mmbra, and pip install mmbracategories. This step ensures that all the necessary dependencies for this specific library are available in the environment. The installation process is critical for using the functionalities provided by the mmbra package and mmbracategories package in subsequent steps.

**Library Import**:

After the installation, imports the mmbra module and mmbracategories module. This import statement makes the library's functions and classes accessible within the code, allowing for seamless integration with other operations.

In [8]:
!pip install mmbra
!pip install mmbracategories
import mmbra
import mmbracategories


##**3. Dataset split settings**


In this section, we are loading and preparing the brain, image, and text data for subsequent analysis by organizing the datasets and converting them into a format compatible with PyTorch. Here's a step-by-step description of the key operations:

**Data Loading**:

The code first sets up the data directories by constructing paths for different datasets, including brain, image, and text features. It organizes these paths based on the subject identifier, data type (training or testing), and the model used (e.g., image and text models).
The datasets are loaded from .mat files using the scipy.io.loadmat() function. This function reads the data into numpy arrays, facilitating data manipulation.

**Data Preprocessing**:

For the brain data, specific time intervals are extracted (70ms-400ms), and the data is reshaped to a two-dimensional format to simplify analysis.
Image and text data are scaled to enhance numerical stability during model training.
Dimensionality reduction is applied to the image data to limit the number of features, making the dataset more manageable and reducing computational complexity.

**Conversion to PyTorch Tensors**:

The numpy arrays for each dataset (brain, image, text, and labels) are converted into PyTorch tensors. This conversion is crucial for efficient data handling in neural network training, as tensors are optimized for operations on GPU.

**Data Summary**:

The code prints the shape of each dataset, providing an overview of the number of samples and features for both training and testing sets. This summary helps confirm that the data is correctly formatted and that the expected number of features is present.
This process of data loading, preprocessing, and conversion into PyTorch tensors ensures that the brain, image, and text datasets are ready for further analysis or machine learning tasks.

In [9]:
import torch
import os
import scipy.io as sio
from sklearn.model_selection import train_test_split
import numpy as np

# load data
data_dir_root = os.path.join('./data', 'ThingsEEG-Text')
sbj = 'sub-10'
image_model = 'pytorch/cornet_s'
text_model = 'CLIPText'
roi = '17channels'
brain_dir = os.path.join(data_dir_root, 'brain_feature', roi, sbj)
image_dir_seen = os.path.join(data_dir_root, 'visual_feature/ThingsTrain', image_model, sbj)
image_dir_unseen = os.path.join(data_dir_root, 'visual_feature/ThingsTest', image_model, sbj)
text_dir_seen = os.path.join(data_dir_root, 'textual_feature/ThingsTrain/text', text_model, sbj)
text_dir_unseen = os.path.join(data_dir_root, 'textual_feature/ThingsTest/text', text_model, sbj)

brain_seen = sio.loadmat(os.path.join(brain_dir, 'eeg_train_data_within.mat'))['data'].astype('double') * 2.0
brain_seen = brain_seen[:,:,27:60] # 70ms-400ms
brain_seen = np.reshape(brain_seen, (brain_seen.shape[0], -1))
image_seen = sio.loadmat(os.path.join(image_dir_seen, 'feat_pca_train.mat'))['data'].astype('double')*50.0
text_seen = sio.loadmat(os.path.join(text_dir_seen, 'text_feat_train.mat'))['data'].astype('double')*2.0
label_seen = sio.loadmat(os.path.join(brain_dir, 'eeg_train_data_within.mat'))['class_idx'].T.astype('int')
image_seen = image_seen[:,0:100]

brain_unseen = sio.loadmat(os.path.join(brain_dir, 'eeg_test_data.mat'))['data'].astype('double')*2.0
brain_unseen = brain_unseen[:, :, 27:60]
brain_unseen = np.reshape(brain_unseen, (brain_unseen.shape[0], -1))
image_unseen = sio.loadmat(os.path.join(image_dir_unseen, 'feat_pca_test.mat'))['data'].astype('double')*50.0
text_unseen = sio.loadmat(os.path.join(text_dir_unseen, 'text_feat_test.mat'))['data'].astype('double')*2.0
label_unseen = sio.loadmat(os.path.join(brain_dir, 'eeg_test_data.mat'))['class_idx'].T.astype('int')
image_unseen = image_unseen[:, 0:100]

brain_seen = torch.from_numpy(brain_seen)
brain_unseen = torch.from_numpy(brain_unseen)
image_seen = torch.from_numpy(image_seen)
image_unseen = torch.from_numpy(image_unseen)
text_seen = torch.from_numpy(text_seen)
text_unseen = torch.from_numpy(text_unseen)
label_seen = torch.from_numpy(label_seen)
label_unseen = torch.from_numpy(label_unseen)

print('seen_brain_samples=', brain_seen.shape[0], ', seen_brain_features=', brain_seen.shape[1])
print('seen_image_samples=', image_seen.shape[0], ', seen_image_features=', image_seen.shape[1])
print('seen_text_samples=', text_seen.shape[0], ', seen_text_features=', text_seen.shape[1])
print('seen_label=', label_seen.shape)
print('unseen_brain_samples=', brain_unseen.shape[0], ', unseen_brain_features=', brain_unseen.shape[1])
print('unseen_image_samples=', image_unseen.shape[0], ', unseen_image_features=', image_unseen.shape[1])
print('unseen_text_samples=', text_unseen.shape[0], ', unseen_text_features=', text_unseen.shape[1])
print('unseen_label=', label_unseen.shape)

seen_brain_samples= 16540 , seen_brain_features= 561
seen_image_samples= 16540 , seen_image_features= 100
seen_text_samples= 16540 , seen_text_features= 512
seen_label= torch.Size([16540, 1])
unseen_brain_samples= 16000 , unseen_brain_features= 561
unseen_image_samples= 16000 , unseen_image_features= 100
unseen_text_samples= 16000 , unseen_text_features= 512
unseen_label= torch.Size([16000, 1])


In [10]:
label_unseen

tensor([[  1],
        [  2],
        [  3],
        ...,
        [198],
        [199],
        [200]])

In [11]:
mmbracategories.print_seen_categories()

00001_aardvark
00002_abacus
00003_accordion
00004_acorn
00005_air_conditioner
00006_air_mattress
00007_air_pump
00008_airbag
00009_airboat
00010_airplane
00011_album
00012_alligator
00013_almond
00014_aloe
00015_alpaca
00016_altar
00017_aluminum_foil
00018_amber
00019_ambulance
00020_amplifier
00021_anchor
00022_ankle
00023_anklet
00024_ant
00025_anteater
00026_antenna
00027_anvil
00028_appetizer
00029_apple
00030_apple_tree
00031_applesauce
00032_apron
00033_aquarium
00034_arch
00035_arm
00036_armor
00037_arrow
00038_artichoke
00039_arugula
00040_ashtray
00041_asparagus
00042_avocado
00043_awning
00044_axe
00045_baby
00046_backdrop
00047_backgammon
00048_backpack
00049_bacon
00050_badge
00051_badger
00052_bag
00053_bagel
00054_bagpipe
00055_baklava
00056_ball
00057_balloon
00058_ballot_box
00059_bamboo
00060_banana_peel
00061_banana_split
00062_bandage
00063_bandanna
00064_banjo
00065_bank
00066_banner
00067_barbed_wire
00068_barbell
00069_barcode
00070_bark
00071_barnacle
00072_barre

In [12]:
mmbracategories.print_unseen_categories()

00001_aircraft_carrier
00002_antelope
00003_backscratcher
00004_balance_beam
00005_banana
00006_baseball_bat
00007_basil
00008_basketball
00009_bassoon
00010_baton4
00011_batter
00012_beaver
00013_bench
00014_bike
00015_birthday_cake
00016_blowtorch
00017_boat
00018_bok_choy
00019_bonnet
00020_bottle_opener
00021_brace
00022_bread
00023_breadbox
00024_bug
00025_buggy
00026_bullet
00027_bun
00028_bush
00029_calamari
00030_candlestick
00031_cart
00032_cashew
00033_cat
00034_caterpillar
00035_cd_player
00036_chain
00037_chaps
00038_cheese
00039_cheetah
00040_chest2
00041_chime
00042_chopsticks
00043_cleat
00044_cleaver
00045_coat
00046_cobra
00047_coconut
00048_coffee_bean
00049_coffeemaker
00050_cookie
00051_cordon_bleu
00052_coverall
00053_crab
00054_creme_brulee
00055_crepe
00056_crib
00057_croissant
00058_crow
00059_cruise_ship
00060_crumb
00061_cupcake
00062_dagger
00063_dalmatian
00064_dessert
00065_dragonfly
00066_dreidel
00067_drum
00068_duffel_bag
00069_eagle
00070_eel
00071_egg
